In [0]:
# SETUP DO NOTEBOOK

"""
%pip install openpyxl
%pip install yfinance
%pip freeze > requirements.txt
"""

import pyspark
import pandas as pd
import yfinance as yf

In [0]:
# INICIO DA BUSCA DOS DADOS

ticker = yf.Ticker("BZ=F")  # Brent
df = ticker.history(period="max")

display(df)

df.to_csv("../Bronze_data/brent_oil.csv")
"""
arquivo combustiveis-liquidos-estados.xlsx baixado de https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/precos-de-distribuicao-de-combustiveis e upado para o databricks manualmente
^^^^
DADOS COLETADOS 09/12/2025
Aparentam ter as medições completas de 09/2020 até meados de 03/2025 a análise tratará os dados no período 09/2020 até 02/2025 para possibilitar um período de análise completo em ambas as bases
"""

# FINALIZAÇÃO DO PRIMEIRO NÍVEL DE DADOS - BRONZE (DADOS BRUTOS)

In [0]:
# INICIO DA TRANSFORMAÇÃO DE DADOS

# TRANSFORMAÇÃO DE DADOS DA BASE DE PETROLEO

df = pd.read_csv("../Bronze_data/brent_oil.csv")

df = df.drop(columns=["Dividends", "Stock Splits"])
df = df.rename(columns={"Date": "date", "Open": "open", "High": "high", "Low": "low", "Close": "close", "Volume": "volume"})

df["date"] = pd.to_datetime(df["date"], utc=True)
df = df[df['date'] < "2025-03-01"]
df = df[df['date'] > "2020-09-01"]

df = df[["date", "open", "high", "low", "close", "volume"]]
df = df.sort_values("date")
df = df.dropna()
df = df.astype({"open": "float64", "high": "float64", "low": "float64", "close": "float64", "volume": "int64"})
df = df.drop_duplicates()

df = (
    df.groupby(df["date"].dt.to_period("M"))
        .agg(
            avg_open=("open", "mean"),
            avg_close=("close", "mean"),
            avg_high=("high", "mean"),
            avg_low=("low", "mean"),
            highest=("high", "max"),
            lowest=("low", "min")
        )
        .reset_index()
)

df["date"] = df["date"].dt.strftime("%Y-%m")

df.to_csv("../Silver_data/brent_oil.csv")

df.display()

In [0]:
# TRANSFORMAÇÃO DE DADOS DA BASE DOS COMBUSTIVEIS

def categorize(prod):
    prod = prod.upper()
    if "GASOLINA" in prod:
        return "GASOLINE"
    if "DIESEL" in prod:
        return "DIESEL"
    if "ETANOL" in prod:
        return "ETHANOL"
    return None

df = pd.read_excel("../Bronze_data/combustiveis-liquidos-estados.xlsx", skiprows=8)

df = df.drop(columns=["UNIDADE DE MEDIDA"])
df = df.rename(columns={"MÊS": "date", "PRODUTO": "product", "REGIÃO": "region", "ESTADO": "state", "PRODUTO": "product", "PREÇO MÉDIO DE DISTRIBUIÇÃO": "price", "DESVIO PADRÃO": "std"})

df["price"] = pd.to_numeric(df["price"], errors="coerce")
df["std"] = pd.to_numeric(df["std"], errors="coerce")

df = df.dropna()
df = df.astype({"price": "float64", "std": "float64"})
df = df.drop_duplicates()

df["date"] = pd.to_datetime(df["date"], errors="coerce")
df = df[df['date'] < "2025-03-01"]
df = df.sort_values("date")

df['product'] = df['product'].apply(categorize)
df = (df.groupby(['date', 'product', 'region', 'state'], as_index=True).agg(avg_price=('price', 'mean'),std=('std', 'mean')))

#df["date"] = df["date"].dt.strftime("%Y-%m")

df.to_csv("../Silver_data/combustiveis.csv")

df.display()

# FINALIZADA A TRANSFORMAÇÃO DOS DADOS AO NÍVEL SILVER (PADRONIZAÇÃO)

In [0]:
# TRANSFORMAÇÃO FINAL DOS DADOS E CRIAÇÃO DA TABELA FATO - GOLD

df_oil = pd.read_csv("../Silver_data/brent_oil.csv")
df_gas = pd.read_csv("../Silver_data/combustiveis.csv")

df = pd.DataFrame()
df['date'] = df_oil['date']
df['oil_avg_price'] = (df_oil['avg_close'] + df_oil['avg_open'] + df_oil['avg_high'] + df_oil['avg_low'])/4
df['oil_pct_variation'] = df['oil_avg_price'].pct_change()

monthly_gas = (
    df_gas
    .groupby(['date', 'product'], as_index=False)
    .agg(
        avg_price=('avg_price', 'mean'),
        std_price=('avg_price', 'std'),
        n_registros=('avg_price', 'count')
    )
)

df['date'] = pd.to_datetime(df['date'], errors="coerce").dt.date

df['gasoline_avg_price'] = monthly_gas.loc[monthly_gas['product'] == 'GASOLINE', 'avg_price'].values
df['gasoline_pct_variation'] = df['gasoline_avg_price'].pct_change()

df['diesel_avg_price'] = monthly_gas.loc[monthly_gas['product'] == 'DIESEL', 'avg_price'].values
df['diesel_pct_variation'] = df['diesel_avg_price'].pct_change()

df['ethanol_avg_price'] = monthly_gas.loc[monthly_gas['product'] == 'ETHANOL', 'avg_price'].values
df['ethanol_pct_variation'] = df['ethanol_avg_price'].pct_change()

df['dif_pct_oil_gasoline'] = df['oil_pct_variation'] - df['gasoline_pct_variation']
df['dif_pct_oil_diesel'] = df['oil_pct_variation'] - df['diesel_pct_variation']
df['dif_pct_oil_ethanol'] = df['oil_pct_variation'] - df['ethanol_pct_variation']

df = df.dropna()

df.to_csv("../Gold_data/tabela_fato.csv")

spark_df = spark.createDataFrame(df)
spark_df_gas = spark.createDataFrame(df_gas)
spark.sql("USE CATALOG `mvp-juj`")
spark.sql("USE SCHEMA `mvp-data`")
spark.sql("DROP TABLE IF EXISTS tabela_fato")
spark.sql("DROP TABLE IF EXISTS combustiveis")
spark_df.write.mode("overwrite").saveAsTable("tabela_fato")
spark_df_gas.write.mode("overwrite").saveAsTable("combustiveis")

df.display()

# FINALIZADA A CRIAÇÃO DA TABELA FATO - GOLD; DADOS PRONTOS PARA ANÁLISE